In [3]:
import pandas as pd
import os

In [2]:
# pip install nltk
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\82105\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
# hugging-face model 불러오기
# pip install transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained('eenzeenee/t5-base-korean-summarization')
tokenizer = AutoTokenizer.from_pretrained('eenzeenee/t5-base-korean-summarization')

c:\Users\82105\AppData\Local\Programs\Python\Python37\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\82105\AppData\Local\Programs\Python\Python37\lib\site-packages\huggingface_hub\file_download.py:129: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\82105\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see acti

In [ ]:
# 코퍼스 키워드로 추출된 공시 데이터 불러오기
path = './1-2. ESG report/'
dir = 'output/'
file_list = os.listdir(path+dir)
# 이중 공시 데이터만 가져옴
file_list = [file for file in file_list if file.startswith("extracted")]

In [ ]:
for file in file_list:
    df = pd.read_csv(file)
    acc = df.drop_duplicates(subset = 'keyword')
    acc['text'] = ''

    for _, row in df.iterrows():
        s = ''
        sub = df[df['keyword']==row['keyword']]
        for _, i in sub.iterrows():
            s += i['text']
        acc.loc[acc['keyword']==row['keyword'],'text'] = s
    

    out = acc.copy()
    out['text'] = ''

    for _, item in acc.iterrows():
        inputs = item['text']
        inputs = tokenizer(inputs, max_length=512, truncation=True, return_tensors="pt")
        output = model.generate(**inputs, num_beams=3, do_sample=True, min_length=10, max_length=128)
        decoded_output = tokenizer.batch_decode(output, skip_special_tokens=False)[0]
        result = nltk.sent_tokenize(decoded_output.strip())[0]
        out.loc[out['keyword']==item['keyword'], 'text'] = result

    # output 폴더 없으면 만들기 
    if not os.path.isdir(path + 'output/'):
        os.mkdir(path + 'output/')
    out.to_csv(path+dir+'summarized_'+file.replace("extracted_", "").replace('.csv',''), index=False, encoding='utf-8-sig')

In [9]:
# file = './1-2. ESG report/output/hyundaehomeshopping_2023.csv'

# df = pd.read_csv(file)
# acc = df.drop_duplicates(subset = 'keyword')
# acc['text'] = ''

# for _, row in acc.iterrows():
#     s = ''
#     sub = df[df['keyword']==row['keyword']]
#     for _, i in sub.iterrows():
#         s += i['text']
#     acc.loc[out['keyword']==row['keyword'],'text'] = s

C:\Users\82105\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
c:\Users\82105\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
c:\Users\82105\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https

In [10]:
# out = acc.copy()
# out['text'] = ''

In [15]:
# for _, item in acc.iterrows():
#     inputs = item['text']
#     inputs = tokenizer(inputs, max_length=512, truncation=True, return_tensors="pt")
#     output = model.generate(**inputs, num_beams=3, do_sample=True, min_length=10, max_length=128)
#     print(output)
#     decoded_output = tokenizer.batch_decode(output, skip_special_tokens=False)[0]
#     print(decoded_output)
#     result = nltk.sent_tokenize(decoded_output.strip())[0]
#     print(result)
#     out.loc[out['keyword']==item['keyword'], 'text'] = result

tensor([[    0,  1369,  4186,   262,   222,  4415,   429,   222,  5374, 30402,
           222,  1588,  5374,   464,   381,   222,  4192,  5374,   464,   222,
           783,   222,  1349,  5374,   464,   293,   222, 10868,   222,   334,
           222,  1813,   222,   480,  2304,   222,  1349,  5374,   464,   222,
         11587,   291,   222,   863,   222,  4128,   515,   222,  3092,   222,
           450,   222,  1349,   222,  1409,   222,  2001,   279,   222,  2430,
         34939,   222,  1177,   333]])
현대홈쇼핑이 배출하는 폐기물은 사무폐기물과 물류폐기물 그리고 방송폐기물로 나눌 수 있으며 당사는 방송폐기물 감축을 위해 미디어월 구축 등 방송 환경 변화에 적극적인 투자를
현대홈쇼핑이 배출하는 폐기물은 사무폐기물과 물류폐기물 그리고 방송폐기물로 나눌 수 있으며 당사는 방송폐기물 감축을 위해 미디어월 구축 등 방송 환경 변화에 적극적인 투자를
tensor([[    0,  1369,  4186,   311,   222,   464,   222, 15585,   222,  2749,
           333,   222,   863,   222,  8095,   222,   417,  2647,   222,  1653,
         35655,   222,  3092,   443,   222,  4328,   222,  5562,   222,  4599,
           333,   222,  2293,   222,  1653,   279,   222, 

KeyboardInterrupt: 